# COT Analysis

#### Install required packages

In [43]:
# !pip install cot-reports

In [44]:
import cot_reports as cot
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt

### Declare important variables for extraction purposes.

In [45]:

'''
special fields for analysing the commitment of traders report
'''
special_columns = [
    "Market and Exchange Names", "As of Date in Form YYYY-MM-DD", "Noncommercial Positions-Long (All)",
    "Noncommercial Positions-Short (All)" , "Change in Noncommercial-Long (All)", 
    "Change in Noncommercial-Short (All)", "Open Interest (All)", "Change in Open Interest (All)"
]


'''
specific markets and exchanges with varying names matching name 
changes over the years
'''
markets_and_exchanges = [ 
    "MEXICAN PESO - CHICAGO MERCANTILE EXCHANGE",
    "MEXICAN PESO - INTERNATIONAL MONETARY MARKET",
    "JAPANESE YEN - CHICAGO MERCANTILE EXCHANGE",
    "JAPANESE YEN - INTERNATIONAL MONETARY MARKET",
    "EURO FX - CHICAGO MERCANTILE EXCHANGE",
    "EURO FX - INTERNATIONAL MONETARY",
    "DJIA Consolidated - CHICAGO BOARD OF TRADE",
    "DOW JONES INDUSTRIAL AVERAGE - CHICAGO BOARD OF TRADE",
    "NASDAQ-100 Consolidated - CHICAGO MERCANTILE EXCHANGE",
    "NASDAQ-100 STOCK INDEX - CHICAGO MERCANTILE EXCHANGE",
    "NASDAQ-100 STOCK INDEX - INTERNATIONAL MONETARY MARKET",
    "NEW ZEALAND DOLLAR - CHICAGO MERCANTILE EXCHANGE",
    "NEW ZEALAND DOLLARS - CHICAGO MERCANTILE EXCHANGE",
    "NEW ZEALAND DOLLARS - INTERNATIONAL MONETARY MARKET",
    "NZ DOLLAR - CHICAGO MERCANTILE EXCHANGE",
    "AUSTRALIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE",
    "AUSTRALIAN DOLLARS - CHICAGO MERCANTILE EXCHANGE",
    "AUSTRALIAN DOLLARS - INTERNATIONAL MONETARY MARKET",
    "CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE",
    "CANADIAN DOLLAR - INTERNATIONAL MONETARY MARKET",
    "SWISS FRANC - CHICAGO MERCANTILE EXCHANGE",
    "SWISS FRANC - INTERNATIONAL MONETARY MARKET",
    "BRITISH POUND STERLING - INTERNATIONAL MONETARY MARKET",
    "BRITISH POUND STERLING - CHICAGO MERCANTILE EXCHANGE",
    "POUND STERLING - CHICAGO MERCANTILE EXCHANGE",
    "POUND STERLING - INTERNATIONAL MONETARY MARKET",
    "BRITISH POUND - CHICAGO MERCANTILE EXCHANGE",
    "U.S. DOLLAR INDEX - NEW YORK COTTON EXCHANGE",
    "U.S. DOLLAR INDEX - ICE FUTURES U.S.",
    "U.S. DOLLAR INDEX - NEW YORK BOARD OF TRADE",
    "USD INDEX - ICE FUTURES U.S.",
    "1000 TROY OUNCE SILVER - CHICAGO BOARD OF TRADE",
    "SILVER - CHICAGO BOARD OF TRADE",
    "SILVER, 5000 TROY OZ - CHICAGO BOARD OF TRADE",
    "SILVER - COMMODITY EXCHANGE INC.",
    "COPPER - COMMODITY EXCHANGE INC.",
    "COPPER-GRADE #1 - COMMODITY EXCHANGE INC.",
    "COPPER- #1 - COMMODITY EXCHANGE INC.",
    "GOLD, 100 TROY OZ - CHICAGO BOARD OF TRADE",
    "GOLD - COMMODITY EXCHANGE INC.",
    "GOLD - INTERNATIONAL MONETARY MARKET",
    "PLATINUM - NEW YORK MERCANTILE EXCHANGE",
    "PLATINUM - COMMODITY EXCHANGE INC.",
]

'''
naming conventions we will adopt
'''
symbol_names = [
    "MEXICAN PESO",
    "JAPANESE YEN",
    "EURO FX",
    "DJIA Consolidated",
    "NASDAQ-100",
    "NEW ZEALAND",
    "AUSTRALIAN",
    "CANADIAN DOLLAR",
    "SWISS FRANC",
    "POUND",
    "U.S. DOLLAR INDEX",
    "SILVER",
    "COPPER",
    "GOLD",
    "PLATINUM",
]

### Get legacy futures from 1986 to 2016 matching the market and exchanges defined above.

In [46]:
# code to download and save futures data.
# legacy_fut = cot.cot_hist(cot_report_type = "legacy_fut", store_txt=True, verbose=True)

'''
save file to csv in directory to speed up future processing
'''
# save legacy_fut to csv for easy load next time
legacy_fut = pd.read_csv("../large files/FUT86_16.txt", low_memory=False)

In [47]:
special_fut = legacy_fut[legacy_fut["Market and Exchange Names"].isin(markets_and_exchanges)].sort_values(by="Market and Exchange Names")

In [48]:
len(special_fut)

19328

In [49]:
special_fut.tail(2)

,Market and Exchange Names,As of Date in Form YYMMDD,As of Date in Form YYYY-MM-DD,CFTC Contract Market Code,CFTC Market Code in Initials,CFTC Region Code,CFTC Commodity Code,Open Interest (All),Noncommercial Positions-Long (All),Noncommercial Positions-Short (All),...,Concentration-Gross LT =8 TDR-Long (Other),Concentration-Gross LT =8 TDR-Short(Other),Concentration-Net LT =4 TDR-Long (Other),Concentration-Net LT =4 TDR-Short (Other),Concentration-Net LT =8 TDR-Long (Other),Concentration-Net LT =8 TDR-Short (Other),Contract Units,CFTC Contract Market Code (Quotes),CFTC Market Code in Initials (Quotes),CFTC Commodity Code (Quotes)
113176,U.S. DOLLAR INDEX - NEW YORK COTTON EXCHANGE,104,2000-01-04,098662,NYCE,1,98,7609,2203,1967,...,0.0,0.0,0.0,0.0,0.0,0.0,(U.S. DOLLAR INDEX X $1000),098662,NYCE,98
113183,U.S. DOLLAR INDEX - NEW YORK COTTON EXCHANGE,991116,1999-11-16,098662,NYCE,1,98,8012,3694,2856,...,97.8,97.8,97.8,97.8,97.8,97.8,(U.S. DOLLAR INDEX X $1000),098662,NYCE,98


In [50]:
# Handle unique case for the DJIA across the entire DataFrame by unifying market and exchange names
special_fut["Market and Exchange Names"]= special_fut["Market and Exchange Names"].replace(
    {
        'DOW JONES INDUSTRIAL AVERAGE - CHICAGO BOARD OF TRADE': 'DJIA Consolidated - CHICAGO BOARD OF TRADE'
    }
)

In [51]:
for sn in symbol_names:
    dfs = []
    results = pd.DataFrame()
    
    # Loop through markets and exchanges and append similar symbols that are similar but with varying names
    # as a result of difference in naming over the years.
    for mx in markets_and_exchanges: 
        if sn in mx:
            dfs.append(special_fut[special_fut["Market and Exchange Names"]==mx])
    
    # Loop through the list and concatenate 
    for df in dfs:
        # Concatenate vertically (along rows)
        results = pd.concat([results, df], axis=0).sort_values(by="As of Date in Form YYYY-MM-DD", ascending=False)
    # Save COT data (special colunmns) of the different symbols in csv.
    results[special_columns].to_csv("data/" + sn + ".csv")

### Get data for legacy futures from 2017 to present

In [52]:
years = np.arange(2017,2025,1)
years

array([2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

### Updating old csv files with new data.

In [53]:
def handle_nzusd_usindex_case(dataframe):
    dataframe["Market and Exchange Names"]= dataframe["Market and Exchange Names"].replace(
        {
            'NZ DOLLAR - CHICAGO MERCANTILE EXCHANGE': 'NEW ZEALAND DOLLAR - CHICAGO MERCANTILE EXCHANGE',
            'USD INDEX - ICE FUTURES U.S.': 'U.S. DOLLAR INDEX - ICE FUTURES U.S.'
        }
    )
    return dataframe

def modify_old_with_new(markets_and_exchanges, symbol_names, dataframe):
    '''
    markets_and_exchanges - as defined above
    symbols_names - as defined above
    dataframe - pandas dataframe containing markets_and_exchanges data for a particular year.
    '''    
    for mx in markets_and_exchanges:
        new_df = dataframe[dataframe["Market and Exchange Names"]==mx][special_columns]
        for sym in symbol_names:
            if sym in mx:
                old_df = pd.read_csv("data/"+sym+".csv")
                merged = pd.concat([old_df, new_df], join='inner', axis=0).sort_values(by="As of Date in Form YYYY-MM-DD", ascending=False)
                merged["Net Positions"] = merged["Noncommercial Positions-Long (All)"]-merged["Noncommercial Positions-Short (All)"]
                merged["Change Net Positions"] = merged["Net Positions"].diff(-1)
                merged.to_csv("data/"+sym+".csv")
            
def update_for_particular_year(year):
    '''
    year - year we want to carry out update on
    '''
    legacy_fut_yr = cot.cot_year(year = year, cot_report_type = "legacy_fut", store_txt=False, verbose=False)
    legacy_fut_yr = legacy_fut_yr[legacy_fut_yr["Market and Exchange Names"].isin(markets_and_exchanges)].sort_values(by="Market and Exchange Names")
    modified_legacy_fut_yr = handle_nzusd_usindex_case(legacy_fut_yr)
    modify_old_with_new(markets_and_exchanges, symbol_names, modified_legacy_fut_yr)

def update_for_multiple_years(years):
    for yr in years:
       update_for_particular_year(yr)

'''
uncomment line to run code for several years.
'''
update_for_multiple_years(years)

### Computing the Open Interest Code for determining price movements for assets

In [54]:
from IPython.display import display, HTML

# Sample Table in HTML
table = """
<table>
  <tr>
    <th>Price</th>
    <th>Open Interest</th>
    <th>Interpretation</th>
    <th>Signal</th>
  </tr>
  <tr>
    <td>Rising</td>
    <td>Rising</td>
    <td>Market is Strong(Bullish)</td>
    <td>1</td>
  </tr>
  <tr>
    <td>Rising</td>
    <td>Falling</td>
    <td>Market is weakening(Bullish Reversal)</td>
    <td>4</td>
  </tr>
  <tr>
    <td>Falling</td>
    <td>Rising</td>
    <td>Market is Weak(Bearish)</td>
    <td>2</td>
  </tr>
  <tr>
    <td>Falling</td>
    <td>Falling</td>
    <td>Market is Strengthing (Bearish Reversal)</td>
    <td>3</td>
  </tr>
</table>
"""

display(HTML(table))

Price,Open Interest,Interpretation,Signal
Rising,Rising,Market is Strong(Bullish),1
Rising,Falling,Market is weakening(Bullish Reversal),4
Falling,Rising,Market is Weak(Bearish),2
Falling,Falling,Market is Strengthing (Bearish Reversal),3


In [55]:
'''
The price of an asset can be deceptive - it doesn't really tell us where it wants to go. 
So modelling the relationship between price and Open interest can be misleading.

Therefore in the above our understanding of price (where price is intented to go) is substituted with the change in Net Positions
We therefore model the relations between Change in Net Position (price) and Open Interest.
'''
def assign_signal_and_interpretation(row):
    if row['Change Net Positions'] > 0 and row['Change in Open Interest (All)'] > 0:
        return 1, 'Market is Strong (Bullish)'
    elif row['Change Net Positions'] > 0 and row['Change in Open Interest (All)'] < 0:
        return 4, 'Market is weakening (Bullish Reversal)'
    elif row['Change Net Positions'] < 0 and row['Change in Open Interest (All)'] > 0:
        return 2, 'Market is Weak (Bearish)'
    elif row['Change Net Positions'] < 0 and row['Change in Open Interest (All)'] < 0:
        return 3, 'Market is Strengthening (Bearish Reversal)'
    else:
        return 0, 'No signal'



In [56]:
symbols_and_tickers = {
    "MEXICAN PESO": "MXNUSD=X",  
    "JAPANESE YEN": "USDJPY=X", 
    "EURO FX": "EURUSD=X", 
    "DJIA Consolidated": "^DJI", 
    "NASDAQ-100": "^NDX", 
    "NEW ZEALAND": "NZDUSD=X", 
    "AUSTRALIAN": "AUDUSD=X",
    "CANADIAN DOLLAR":"CADUSD=X", 
    "SWISS FRANC": "USDCHF=X", 
    "POUND": "GBPUSD=X", 
    "U.S. DOLLAR INDEX": "DX-Y.NYB", 
    "SILVER": "SI=F", 
    "COPPER": "HG=F", 
    "GOLD": "GC=F",
    "PLATINUM": "PL=F" 
}

In [57]:
def toDatetime(dataframe, columnName):
    # Use copy to avoid SettingWithCopyWarning 
    dataframe = dataframe.copy()
    
    dataframe[columnName] = pd.to_datetime(dataframe[columnName])
    return dataframe

def getAsset(filename, path="data/"):
    '''
    filename - asset symbol name dot(.) csv
    '''
    asset = pd.read_csv(path + filename + ".csv").drop(columns="Unnamed: 0")
    return asset

def getSymbolDataYahooFinance(symbolName, s="1986-01-01", i="1d", x=dt.datetime.now()):
    return yf.download(
        symbolName,
        start=s,
        end=str(x.year) + "-" + str(x.month) + "-" + str(x.day),
        interval=i,
        auto_adjust=None,
    ).reset_index()

def saveClosingPrice():
     for key in symbols_and_tickers:
        # Fetch data from Yahoo Finance
        data = getSymbolDataYahooFinance(symbols_and_tickers[key])
        data = data.rename(columns={'Date': 'As of Date in Form YYYY-MM-DD'})

        # Convert date column to datetime
        new_df = toDatetime(data[['As of Date in Form YYYY-MM-DD', 'Close']], 'As of Date in Form YYYY-MM-DD')

        new_df = new_df.copy()
        
        # Calculate price change: use copy to avoid SettingWithCopyWarning
        new_df["Change Price"] = new_df["Close"].diff(-1)
        
        # Load asset data
        asset = toDatetime(getAsset(key), 'As of Date in Form YYYY-MM-DD')

        # flatten multilevel dataframes
        new_df.columns = [col[0] if isinstance(col, tuple) else col for col in new_df.columns]
        # Merge asset data and closing price
        # Explicitly specify the merge key and ensure consistent data types
        return pd.merge(
            asset, 
            new_df,
            how="inner",
            on="As of Date in Form YYYY-MM-DD"
        )
         
# Main operations

def perform_signal():
    for key in symbols_and_tickers:
        # Load asset data
        asset = toDatetime(getAsset(key), 'As of Date in Form YYYY-MM-DD')
        
        # Replace non-numeric values with NaN, then convert to int
        asset.loc[:, 'Change in Open Interest (All)'] = (
            pd.to_numeric(asset['Change in Open Interest (All)'], errors='coerce')
            .fillna(0)
            .astype(int)
        )
    
        # Apply the function 2 to each row of the DataFrame and create 'signal' and 'Interpretation' columns
        asset[['signal', 'Interpretation']] = asset.apply(
            assign_signal_and_interpretation,
            axis=1,
            result_type="expand"
        )
        
        # Save the result to a CSV file
        asset.to_csv("signal/" + key + ".csv", index=False)

In [58]:
# save signal information with/without closing prices.
perform_signal()

## Update data for the current year and apply Open interest code

In [59]:
update_for_particular_year(2025)
perform_signal()

## The End

**References**

- [COT Reporthttps: //www.investopedia.com/terms/c/cot.asp](https://www.investopedia.com/terms/c/cot.asp)